In [1]:
import pandas as pd
import os

# ==========================================
# 1. CONFIGURAÇÃO
# ==========================================
# Vamos usar o RAW (Excel) porque precisamos dos nomes completos das perguntas
# O CSV processado pode ter simplificado os nomes, e precisamos do texto da pergunta para o relatório.
RAW_DATA_PATH = '../data/raw/ebt_municipios_br.xlsx'

print("🔄 Carregando dados brutos para auditoria detalhada...")
df = pd.read_excel(RAW_DATA_PATH)

# Limpeza básica dos nomes das colunas (retirar espaços nas pontas)
df.columns = [c.strip() for c in df.columns]

# ==========================================
# 2. ISOLAR O ALVO (PARACAMBI)
# ==========================================
# Mapeando a coluna de município novamente
col_municipio = next((c for c in df.columns if c.lower() in ['municipio', 'entidade avaliada', 'nome_municipio']), None)
col_uf = next((c for c in df.columns if c.lower() in ['uf', 'estado', 'sigla_uf']), None)

# Filtra Paracambi no RJ
alvo = df[
    (df[col_uf].str.strip() == 'Rio de Janeiro') & 
    (df[col_municipio].astype(str).str.contains('Paracambi'))
].copy()

if alvo.empty:
    raise ValueError("❌ Paracambi não encontrada. Verifique os filtros.")

print(f"✅ Dados de Paracambi carregados. Nota Geral: {alvo['Nota Avaliação'].values[0]}")

# ==========================================
# 3. AUDITORIA DE PONTOS PERDIDOS (A MÁGICA)
# ==========================================
print("\n🕵️‍♂️ Iniciando varredura de vulnerabilidades...")

report_gaps = []

# Iterar sobre as colunas. 
# A lógica da CGU geralmente é: Coluna "Pergunta" seguida imediatamente por "Nota Questão"
# Vamos procurar colunas que tenham "Nota" no nome e verificar se o valor é 0.

for i, col_name in enumerate(df.columns):
    # Se a coluna atual é uma nota...
    if 'Nota' in col_name and 'Avaliação' not in col_name: # Ignora a Nota Final
        nota_obtida = alvo[col_name].values[0]
        
        # Se a nota for ZERO ou NULA, é um problema grave
        if nota_obtida == 0 or pd.isna(nota_obtida):
            
            # Tenta pegar o nome da pergunta (geralmente é a coluna imediatamente anterior)
            # Ex: Coluna 10 é a Pergunta, Coluna 11 é a Nota.
            pergunta_provavel = df.columns[i-1]
            
            # Adiciona ao relatório
            report_gaps.append({
                'Item Avaliado (Pergunta)': pergunta_provavel,
                'Nota Obtida': 0,
                'Status': '🔴 CRÍTICO (Ação Imediata)',
                'Coluna Técnica': col_name
            })

# Transformar em DataFrame para visualização bonita
df_gaps = pd.DataFrame(report_gaps)

# ==========================================
# 4. EXIBIÇÃO DO PLANO DE AÇÃO
# ==========================================
pd.set_option('display.max_colwidth', None) # Mostra o texto todo da pergunta

print(f"\n📋 RELATÓRIO DE FALHAS IDENTIFICADAS: {len(df_gaps)} ITENS")
print("Este é o seu 'Lead': A lista exata do que eles precisam consertar no site.")

if not df_gaps.empty:
    # Mostra tabela formatada
    display(df_gaps[['Item Avaliado (Pergunta)', 'Status']])
    
    # Salva em Excel para você mandar pro cliente se quiser
    df_gaps.to_excel('../data/processed/plano_acao_paracambi.xlsx', index=False)
    print("\n💾 Relatório salvo em: data/processed/plano_acao_paracambi.xlsx")
else:
    print("🎉 Incrível! Nenhum item zerado encontrado (o que contradiz a nota 6.11, verifique os dados).")

# ==========================================
# FIM
# ==========================================

🔄 Carregando dados brutos para auditoria detalhada...
✅ Dados de Paracambi carregados. Nota Geral: 6.11

🕵️‍♂️ Iniciando varredura de vulnerabilidades...

📋 RELATÓRIO DE FALHAS IDENTIFICADAS: 30 ITENS
Este é o seu 'Lead': A lista exata do que eles precisam consertar no site.


,Item Avaliado (Pergunta),Status
0,1.1 - Informe o link do sítio oficial:,🔴 CRÍTICO (Ação Imediata)
1,2.1 - Informe o link do portal de transparência:,🔴 CRÍTICO (Ação Imediata)
2,4.1 - Indique as informações sobre as unidades administrativas que o ente federado disponibiliza:,🔴 CRÍTICO (Ação Imediata)
3,5.1 - Indique as informações localizadas sobre Receitas:,🔴 CRÍTICO (Ação Imediata)
4,6.1 - Indique as informações localizadas sobre Empenhos:,🔴 CRÍTICO (Ação Imediata)
5,6.2 - Indique as informações localizadas sobre Pagamentos:,🔴 CRÍTICO (Ação Imediata)
6,9.1 - Indique as informações localizadas sobre Licitações:,🔴 CRÍTICO (Ação Imediata)
7,12.1 - Indique as informações localizadas sobre Contratos:,🔴 CRÍTICO (Ação Imediata)
8,12.2 - O ente federado disponibiliza o conteúdo integral dos contratos?,🔴 CRÍTICO (Ação Imediata)
9,Indique as informações localizadas sobre Obras Públicas:,🔴 CRÍTICO (Ação Imediata)



💾 Relatório salvo em: data/processed/plano_acao_paracambi.xlsx


In [7]:
import pandas as pd
import numpy as np
import os

# ==========================================
# 1. CARGA E CONVERSÃO ROBUSTA
# ==========================================
RAW_DATA_PATH = '../data/raw/ebt_municipios_br.xlsx'

print("🔄 Carregando dados brutos...")
# Lê o Excel e força tudo a ser string inicialmente para tratarmos as vírgulas manualmente
df = pd.read_excel(RAW_DATA_PATH, dtype=str)

# Limpeza de nomes de colunas
df.columns = [c.strip() for c in df.columns]

# Identificadores
col_municipio = next((c for c in df.columns if c.lower() in ['municipio', 'entidade avaliada', 'nome_municipio']), None)
col_uf = next((c for c in df.columns if c.lower() in ['uf', 'estado', 'sigla_uf']), None)
col_nota_final = next((c for c in df.columns if 'final' in c.lower() or 'avaliação' in c.lower()), None)

# ==========================================
# 2. SANITIZAÇÃO DE NÚMEROS (O Segredo)
# ==========================================
print("🧹 Convertendo notas de texto (0,50) para números (0.50)...")

# Função para corrigir números brasileiros
def fix_brazilian_number(val):
    if pd.isna(val): return 0.0
    val = str(val).replace(',', '.') # Troca vírgula por ponto
    try:
        return float(val)
    except:
        return 0.0

# Aplica a correção em TODAS as colunas que têm "Nota" no nome
cols_notas = [c for c in df.columns if 'Nota' in c]
for col in cols_notas:
    df[col] = df[col].apply(fix_brazilian_number)

# Também corrige a Nota Final para garantir
if col_nota_final:
    df[col_nota_final] = df[col_nota_final].apply(fix_brazilian_number)

# ==========================================
# 3. FILTRAR ESTADO E ALVO
# ==========================================
df_rj = df[df[col_uf].str.strip() == 'Rio de Janeiro'].copy()
print(f"📍 Base RJ carregada: {df_rj.shape[0]} municípios.")

alvo = df_rj[df_rj[col_municipio].str.contains('Paracambi', na=False)].copy()

if alvo.empty:
    raise ValueError("❌ Paracambi não encontrada.")

print(f"✅ Alvo: {alvo[col_municipio].values[0]} | Nota Geral: {alvo[col_nota_final].values[0]}")

# ==========================================
# 4. GERAÇÃO DE LEADS (AUDITORIA INTELIGENTE)
# ==========================================
print("\n🕵️‍♂️ Cruzando dados para encontrar Gaps REAIS...")

comparativo_list = []

for i, col_nota in enumerate(df.columns):
    # Só analisa colunas de Nota (excluindo a nota geral final)
    if 'Nota' in col_nota and col_nota != col_nota_final:
        
        # 1. Descobre a nota de Paracambi e a Máxima do RJ
        nota_paracambi = alvo[col_nota].values[0]
        max_nota_rj = df_rj[col_nota].max()
        
        # 2. A LÓGICA DE OURO:
        # Só é problema se:
        # a) Paracambi tirou 0 (ou muito baixo)
        # b) ALGUÉM no RJ tirou nota maior (max_nota_rj > 0). 
        #    (Se o max for 0, a questão não vale ponto, então ignoramos)
        
        if nota_paracambi == 0 and max_nota_rj > 0:
            
            # Pega a pergunta (coluna anterior)
            pergunta = df.columns[i-1]
            
            # Descobre quem é a referência (quem tirou a nota máxima)
            quem_gabaritou = df_rj[df_rj[col_nota] == max_nota_rj]
            # Tenta pegar alguém que não seja a própria capital ou estado, se possível, pra ser mais "realista"
            exemplo = quem_gabaritou[~quem_gabaritou[col_municipio].str.contains('Governo|Capital', regex=True)]
            
            if exemplo.empty:
                nome_exemplo = quem_gabaritou.iloc[0][col_municipio]
            else:
                nome_exemplo = exemplo.iloc[0][col_municipio]
            
            comparativo_list.append({
                'Questão Chave': pergunta,
                'Nota Paracambi': 0.0,
                'Nota Máxima Possível': max_nota_rj,
                'Referência (Benchmarking)': nome_exemplo
            })

# ==========================================
# 5. RESULTADO FINAL
# ==========================================
df_gap = pd.DataFrame(comparativo_list)

if not df_gap.empty:
    # Ordena pelas notas maiores (Onde estamos perdendo mais ponto?)
    df_gap = df_gap.sort_values(by='Nota Máxima Possível', ascending=False)
    
    print(f"\n📋 ENCONTRADOS {len(df_gap)} PONTOS DE MELHORIA REAIS.")
    print("Estas são as funcionalidades que FALTAM no site e que valem nota.")
    
    pd.set_option('display.max_colwidth', None)
    display(df_gap.head(15)) # Mostra os top 15 erros
    
    # Salvar
    df_gap.to_excel('../data/processed/relatorio_final_gaps.xlsx', index=False)
    print("\n💾 Relatório salvo: data/processed/relatorio_final_gaps.xlsx")
else:
    print("🎉 Nada encontrado. Se a nota não é 10, revise a lógica das colunas.")

🔄 Carregando dados brutos...
🧹 Convertendo notas de texto (0,50) para números (0.50)...
📍 Base RJ carregada: 39 municípios.
✅ Alvo: Paracambi - RJ | Nota Geral: 6.11

🕵️‍♂️ Cruzando dados para encontrar Gaps REAIS...

📋 ENCONTRADOS 6 PONTOS DE MELHORIA REAIS.
Estas são as funcionalidades que FALTAM no site e que valem nota.


,Questão Chave,Nota Paracambi,Nota Máxima Possível,Referência (Benchmarking)
4,24 - PEDIDO 2: O pedido 2 foi enviado com sucesso e obteve resposta?,0.0,1.10,Niterói - RJ
5,25 - PEDIDO 3: O pedido 3 foi enviado com sucesso e obteve resposta?,0.0,1.10,Campos dos Goytacazes - RJ
0,11 - O ente federado disponibiliza consulta para acesso aos resultados das licitações ocorridas?,0.0,0.30,Rio de Janeiro
2,14 - OBRAS PÚBLICAS: O ente federado disponibiliza consulta para o acompanhamento de Obras Públicas?,0.0,0.25,Mesquita - RJ
1,13 - O ente federado permite gerar relatório da consulta de licitações ou da consulta de contratos em formato aberto?,0.0,0.10,Rio de Janeiro
3,19 - BASES DE DADOS ABERTOS: O ente federado publica em seu sítio oficial alguma relação das bases dedados abertos do município (catálago/inventário de dados abertos)?,0.0,0.05,Macaé - RJ



💾 Relatório salvo: data/processed/relatorio_final_gaps.xlsx
